In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import scipy.special as special 
from scipy.optimize import curve_fit 
import seaborn as sns 
from sklearn.metrics import r2_score 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#Cargar desde un archivo .csv
Mexico = pd.read_csv('México.csv')

#Eliminar columnas inecesarias
df = Mexico.drop(['Unnamed: 0'], axis=1)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26582 entries, 0 to 26581
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   26582 non-null  int64  
 1   last_scraped                 26582 non-null  object 
 2   source                       26582 non-null  object 
 3   name                         26582 non-null  object 
 4   description                  26582 non-null  object 
 5   neighborhood_overview        26582 non-null  object 
 6   host_name                    26582 non-null  object 
 7   host_since                   26582 non-null  object 
 8   host_location                26582 non-null  object 
 9   host_about                   26582 non-null  object 
 10  host_response_time           26582 non-null  object 
 11  host_is_superhost            26582 non-null  object 
 12  host_verifications           26582 non-null  object 
 13  host_identity_ve

In [56]:
# df["availability_365"] = pd.cut(df["availability_365"], 
#                                 bins=[-1, 182, 365], 
#                                 labels=["menosDeMedioAño", "masDeMedioAño"])

# df["host_total_listings_count"] = pd.cut(df["host_total_listings_count"], 
#                                 bins=[-1, 65, 550 ], 
#                                 labels=["De0-65", "De66-550"]) 


# Tercer modelo beds
# df["beds"] = df["beds"].replace([0, 1, 1.8, 2], "menorQueTres")
# df["beds"] = df["beds"].replace([3, 4, 5, 6], "IgualOmayorQueTres")

#Cuarto modelo bedrooms
# df["bedrooms"] = df["bedrooms"].replace([0, 1, 1.5, 1.6], "menorDeDos")
# df["bedrooms"] = df["bedrooms"].replace([2, 3, 4, 5, 6], "igualOMayorDeDos")

# Quinto modelo bathrooms
# df["bathrooms"] = df["bathrooms"].replace([0, 0.5, 1, 1.4, 1.5, 2], "menorOIgualDeDos")
# df["bathrooms"] = df["bathrooms"].replace([2.5, 3, 3.5, 4, 4.5], "MayorDeDos")

# Octavo modelo bathrooms
# df["host_verifications"] = df["host_verifications"].replace(["['email', 'phone', 'work_email']", "['email', 'phone']", "['email', 'work_email']", "['phone', 'work_email']"], "dosOMas")
# df["host_verifications"] = df["host_verifications"].replace(["['email']", "['phone']", "['work_email']", '[]'], "ceroOUno")

# decimo modelo bathrooms
df["room_type"] = df["room_type"].replace(["Entire home/apt", "Hotel room"], "EntireH&HotelRoom")
df["room_type"] = df["room_type"].replace(["Private room", "Shared room"], "Private&SharedRoom")

# Comprbar valores unicos
print(np.unique(df["room_type"]))


['EntireH&HotelRoom' 'Private&SharedRoom']


# Primer modelo: Host_Is_Superhost

In [3]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "beds", "reviews_per_month", "accommodates"]]
# dichotomic variables
varsDep = df["host_is_superhost"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

In [5]:
from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [6]:
print(algoritmo.coef_)

[[-0.40151591  0.18490158 -0.03756775  0.43155002  0.18662757]]


In [7]:
y_pred = np.where(algoritmo.predict_proba(X_test)[:,1] > 0.3, 't', 'f')
y_pred

print(np.unique(y_pred))

['f' 't']


In [8]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz")
print(matriz)

Matriz
[[1895 3142]
 [ 609 2329]]


In [9]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print("Precision del modelo para f")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print("Precision del modelo para t")
print(precision)

Precision del modelo para f
0.7567891373801917
Precision del modelo para t
0.4256991409248766


In [10]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

Exactitud
0.5296551724137931


In [11]:
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print("sensibilidad")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("sensibilidad")
print(sensibilidad)

sensibilidad
0.376216001588247
sensibilidad
0.7927161334240981


# Segundo Modelo HostIdentityVerified

In [12]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_curve
from imblearn.over_sampling import SMOTE

# Variables independientes y dependientes
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "beds", "reviews_per_month", "accommodates"]]
varsDep = df["host_identity_verified"]

# Creación de X e y
X = varsIndep
y = varsDep

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Verificar balance de clases
print("Distribución original de y_train:")
print(y_train.value_counts())

# Aplicar SMOTE para balancear
smote = SMOTE(sampling_strategy=0.3, random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Verificar balance después de SMOTE
print("Distribución después de SMOTE:")
print(pd.Series(y_train_balanced).value_counts())

# Normalización
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Modelo
modelo = LogisticRegression()
modelo.fit(X_train_balanced, y_train_balanced)

# Encontrar el mejor umbral
probs = modelo.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs, pos_label="t")
best_threshold = thresholds[np.argmax(tpr - fpr)]

# Predicción con el mejor umbral
y_pred = np.where(probs > best_threshold, 't', 'f')

# Evaluación
print("Precisión para 'f':", precision_score(y_test, y_pred, average="binary", pos_label="f"))
print("Precisión para 't':", precision_score(y_test, y_pred, average="binary", pos_label="t"))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad para 'f':", recall_score(y_test, y_pred, average="binary", pos_label="f"))
print("Sensibilidad para 't':", recall_score(y_test, y_pred, average="binary", pos_label="t"))

Distribución original de y_train:
host_identity_verified
t    17795
f      812
Name: count, dtype: int64
Distribución después de SMOTE:
host_identity_verified
t    17795
f     5338
Name: count, dtype: int64
Precisión para 'f': 0.13559322033898305
Precisión para 't': 0.9768257356304315
Exactitud: 0.827460815047022
Sensibilidad para 'f': 0.5581395348837209
Sensibilidad para 't': 0.8396016249508583


# Tercer Modelo BEDS

In [16]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates"]]
# dichotomic variables
varsDep = df["beds"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="IgualOmayorQueTres")
print("Precision del modelo IgualOmayorQueTres")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="menorQueTres")
print("Precision del modelo menorQueTres")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="IgualOmayorQueTres")
print("sensibilidad IgualOmayorQueTres")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="menorQueTres")
print("sensibilidad menorQueTres")
print(sensibilidad)

['IgualOmayorQueTres' 'menorQueTres']
Precision del modelo IgualOmayorQueTres
0.7884187082405345
Precision del modelo menorQueTres
0.900522820404126
Exactitud
0.8878996865203762
sensibilidad IgualOmayorQueTres
0.5014164305949008
sensibilidad menorQueTres
0.9710498247752553


# Cuarto modelo  BEDROOMS

In [23]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "beds"]]
# dichotomic variables
varsDep = df["bedrooms"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="igualOMayorDeDos")
print("Precision del modelo igualOMayorDeDos")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="menorDeDos")
print("Precision del modelo menosDeDos")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="igualOMayorDeDos")
print("sensibilidad igualOMayorDeDos")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="menorDeDos")
print("sensibilidad menosDeDos")
print(sensibilidad)

['igualOMayorDeDos' 'menorDeDos']
Precision del modelo igualOMayorDeDos
0.8370468905886266
Precision del modelo menosDeDos
0.9229066022544283
Exactitud
0.8905329153605016
sensibilidad igualOMayorDeDos
0.8679310344827587
sensibilidad menosDeDos
0.903448275862069


# Quinto modelo BATHROOMS

In [29]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "beds", "bedrooms"]]
# dichotomic variables
varsDep = df["bathrooms"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="MayorDeDos")
print("Precision del modelo MayorDeDos")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="menorOIgualDeDos")
print("Precision del modelo menorOIgualDeDos")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="MayorDeDos")
print("sensibilidad MayorDeDos")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="menorOIgualDeDos")
print("sensibilidad menorOIgualDeDos")
print(sensibilidad)

['MayorDeDos' 'menorOIgualDeDos']
Precision del modelo MayorDeDos
0.6225165562913907
Precision del modelo menorOIgualDeDos
0.9336656441717791
Exactitud
0.9277742946708464
sensibilidad MayorDeDos
0.1533442088091354
sensibilidad menorOIgualDeDos
0.9922575387123065


# Sexto modelo instant bookable

In [30]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "beds", "bedrooms"]]
# dichotomic variables
varsDep = df["instant_bookable"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print("Precision del modelo t")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print("Precision del modelo f")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print("sensibilidad t")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print("sensibilidad f")
print(sensibilidad)

['f' 't']
Precision del modelo t
0.5657370517928287
Precision del modelo f
0.6302083333333334
Exactitud
0.6200626959247649
sensibilidad t
0.2222222222222222
sensibilidad f
0.8859832635983264


# septimo modelo Availability365

In [36]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "beds", "bedrooms"]]
# dichotomic variables
varsDep = df["availability_365"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="masDeMedioAño")
print("Precision del modelo masDeMedioAño")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="menosDeMedioAño")
print("Precision del modelo menosDeMedioAño")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="masDeMedioAño")
print("sensibilidad masDeMedioAño")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="menosDeMedioAño")
print("sensibilidad menosDeMedioAño")
print(sensibilidad)

['masDeMedioAño' 'menosDeMedioAño']
Precision del modelo masDeMedioAño
0.6115015974440895
Precision del modelo menosDeMedioAño
0.4
Exactitud
0.6075235109717868
sensibilidad masDeMedioAño
0.9815384615384616
sensibilidad menosDeMedioAño
0.01935483870967742


# Octavo modelo Host verifications

In [44]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_curve
from imblearn.over_sampling import SMOTE

# Variables independientes y dependientes
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "beds", "reviews_per_month", "accommodates"]]
varsDep = df["host_verifications"]

# Creación de X e y
X = varsIndep
y = varsDep

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Verificar balance de clases
print("Distribución original de y_train:")
print(y_train.value_counts())

# Aplicar SMOTE para balancear
smote = SMOTE(sampling_strategy=0.3, random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Verificar balance después de SMOTE
print("Distribución después de SMOTE:")
print(pd.Series(y_train_balanced).value_counts())

# Normalización
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Modelo
modelo = LogisticRegression()
modelo.fit(X_train_balanced, y_train_balanced)

# Encontrar el mejor umbral
probs = modelo.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs, pos_label="dosOMas")
best_threshold = thresholds[np.argmax(tpr - fpr)]

# Predicción con el mejor umbral
y_pred = np.where(probs > best_threshold, 'dosOMas', 'ceroOUno')

# Evaluación
print("Precisión para 'dosOMas':", precision_score(y_test, y_pred, average="binary", pos_label="dosOMas"))
print("Precisión para 'ceroOUno':", precision_score(y_test, y_pred, average="binary", pos_label="ceroOUno"))
print("Exactitud:", accuracy_score(y_test, y_pred))
print("Sensibilidad para 'dosOMas':", recall_score(y_test, y_pred, average="binary", pos_label="dosOMas"))
print("Sensibilidad para 'ceroOUno':", recall_score(y_test, y_pred, average="binary", pos_label="ceroOUno"))

Distribución original de y_train:
host_verifications
dosOMas     16802
ceroOUno     1805
Name: count, dtype: int64
Distribución después de SMOTE:
host_verifications
dosOMas     16802
ceroOUno     5040
Name: count, dtype: int64
Precisión para 'dosOMas': 0.9212541013488881
Precisión para 'ceroOUno': 0.10894495412844037
Exactitud: 0.3883385579937304
Sensibilidad para 'dosOMas': 0.35150925024342744
Sensibilidad para 'ceroOUno': 0.7251908396946565


# Noveno modelo host_total_listings_count

In [51]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "host_listings_count", "bedrooms"]]
# dichotomic variables
varsDep = df["host_total_listings_count"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="De0-65")
print("Precision del modelo De0-65")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="De66-550")
print("Precision del modelo De66-550")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="De0-65")
print("sensibilidad De0-65")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="De66-550")
print("sensibilidad De66-550")
print(sensibilidad)

['De0-65' 'De66-550']
Precision del modelo De0-65
0.9501578116780641
Precision del modelo De66-550
0.8005390835579514
Exactitud
0.9431974921630094
sensibilidad De0-65
0.9898616248801205
sensibilidad De66-550
0.4393491124260355


# Decimo Modelo room_type

In [57]:
varsIndep = df[["minimum_nights", "minimum_nights_avg_ntm", "reviews_per_month", "accommodates", "host_listings_count", "bedrooms"]]
# dichotomic variables
varsDep = df["room_type"]

# Creation of X and y variables for model
X= varsIndep 
y= varsDep

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

from sklearn. linear_model import LogisticRegression

escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

print(np.unique(y_pred))

from sklearn.metrics import precision_score

#Precision
#Change to fit the actual values of the varsdep variable
precision = precision_score(y_test, y_pred, average="binary", pos_label="EntireH&HotelRoom")
print("Precision del modelo EntireH&HotelRoom")
print(precision)

precision = precision_score(y_test, y_pred, average="binary", pos_label="Private&SharedRoom")
print("Precision del modelo Private&SharedRoom")
print(precision)


#Exactitud
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud")
print(exactitud)

#Sensibilidad
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="EntireH&HotelRoom")
print("sensibilidad EntireH&HotelRoom")
print(sensibilidad)
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Private&SharedRoom")
print("sensibilidad Private&SharedRoom")
print(sensibilidad)

['EntireH&HotelRoom' 'Private&SharedRoom']
Precision del modelo EntireH&HotelRoom
0.8511504785176135
Precision del modelo Private&SharedRoom
0.6546997389033943
Exactitud
0.7756739811912226
sensibilidad EntireH&HotelRoom
0.7980145093547155
sensibilidad Private&SharedRoom
0.7329192546583851
